In [133]:
import pandas as pd # type: ignore

# Read the CSV file
data = pd.read_csv('train.csv')

# Display the first few rows of the data
data.head()
data.columns = data.columns.str.replace("'", "", regex=False)

In [134]:
def extract_Location(data_set: pd.DataFrame) -> pd.DataFrame:
    data_set['Location'] = data_set['Location'].str.extract(r'([^-\s]+)')
    data_set['Location'].fillna('NA', inplace=True)
    return data_set
data= extract_Location(data)

/tmp/ipykernel_35404/2643891686.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_set['Location'].fillna('NA', inplace=True)


In [135]:
def extract_date(data_set: pd.DataFrame) -> pd.DataFrame:
    data_set['Publication_Date'] = data_set['Publication_Date'].str.extract(r'Publicado el (.+)')
    data_set['Publication_Date'] = pd.to_datetime(data_set['Publication_Date'], errors='coerce')
    data_set['Publication_Date'] = data_set['Publication_Date'].fillna(data_set['Publication_Date'].mode()[0])
    data_set['Publication_Year'] = data_set['Publication_Date'].dt.year
    data_set['Publication_Month'] = data_set['Publication_Date'].dt.month
    data_set['Publication_Day'] = data_set['Publication_Date'].dt.day
    data_set.drop(columns='Publication_Date', inplace=True)
    return data_set

In [136]:
data = extract_date(data)

/tmp/ipykernel_35404/2842108097.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_set['Publication_Date'] = pd.to_datetime(data_set['Publication_Date'], errors='coerce')


In [137]:
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')

In [138]:
data.groupby('Garden').size()

Garden
0                 906
1                3384
5+                  3
Not specified    2210
dtype: int64

In [139]:
data.groupby('Bedrooms').size()

Bedrooms
 CaÃ±ete            3
0                  10
1                  31
2                 145
3                1536
4                2541
5+               2206
Not specified      31
dtype: int64

In [140]:
for column in data.columns:
    print('--------------------------------')
    print(data.groupby(column).size())

--------------------------------
Id
0       1
1       1
2       1
3       1
4       1
       ..
6995    1
6996    1
6997    1
6998    1
6999    1
Length: 7000, dtype: int64
--------------------------------
Gas_Connection
0                3175
1                 484
Not specified    3341
dtype: int64
--------------------------------
Fireplace
0                3270
1                1319
Not specified    2411
dtype: int64
--------------------------------
Entrance_Hall
0                1864
1                2695
Not specified    2441
dtype: int64
--------------------------------
Kitchenette
0                6791
Not specified     209
dtype: int64
--------------------------------
Equipped
0                4161
1                 428
Not specified    2411
dtype: int64
--------------------------------
Garage_Type
Lineales          604
NoTiene           835
Not specified    2892
Paralelas        2505
Separadas         164
dtype: int64
--------------------------------
Furnished
0                3

In [141]:
# Remove Construction_Area column, its redundant with construction area
data.drop(columns=['Construction_Area'], inplace=True)


In [142]:
data['Total_Area_m2'] = pd.to_numeric(data['Total_Area_m2'], errors='coerce')

In [143]:
dict_bedrooms = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5+': 5
}
def preprocess_data(data_set):
    data_set['Bedrooms'] = data_set['Bedrooms'].apply(lambda x: dict_bedrooms.get(x, x))
    data_set['Bedrooms'] = data_set['Bedrooms'].apply(lambda x: x if isinstance(x, (int, float)) else 0)
    data_set['Bedrooms'].fillna(data_set['Bedrooms'].mode()[0], inplace=True)
    return data_set

In [144]:
data = preprocess_data(data)

/tmp/ipykernel_35404/2115012877.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_set['Bedrooms'].fillna(data_set['Bedrooms'].mode()[0], inplace=True)


In [145]:
numeric_columns = ['Number_Floors', 'Age']
def fill_numeric_column(column_name, dataset):
    dataset[column_name] = pd.to_numeric(dataset[column_name], errors='coerce')
    dataset[column_name].fillna(dataset[column_name].mode()[0], inplace=True)
    return dataset

In [146]:

for column in numeric_columns:
    data = fill_numeric_column(column, data)

/tmp/ipykernel_35404/1612466116.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column_name].fillna(dataset[column_name].mode()[0], inplace=True)


In [147]:
data

,Id,Gas_Connection,Fireplace,Entrance_Hall,Kitchenette,Equipped,Garage_Type,Furnished,Drainage,Telephone,...,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom,Price,Publication_Year,Publication_Month,Publication_Day
0,0,1,Not specified,0,0,Not specified,Paralelas,Not specified,Not specified,0,...,NaN,NaN,NaN,NaN,NaN,NaN,210000,2019,9,18
1,1,0,1,Not specified,0,0,Paralelas,0,0,0,...,0,LaMolina,Not specified,Bueno,Not specified,1,660000,2019,4,9
2,2,Not specified,Not specified,1,0,Not specified,Lineales,Not specified,Not specified,Not specified,...,Not specified,LaMolina,0,Bueno,0,1,1500000,2019,1,8
3,3,0,1,Not specified,0,0,Separadas,0,0,0,...,0,SanIsidro,Not specified,Remodelado,Not specified,1,1100000,2019,12,8
4,4,0,Not specified,Not specified,0,Not specified,Lineales,Not specified,Not specified,0,...,Not specified,VillaMariaDelTriunfo,Not specified,Regular,Not specified,0,110000,2019,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,6995,Not specified,Not specified,Not specified,0,Not specified,Not specified,Not specified,Not specified,Not specified,...,Not specified,LaVictoria,Not specified,Bueno,Not specified,0,450000,2019,7,26
6996,6996,0,0,1,0,0,Not specified,1,0,0,...,0,Asia,0,Not specified,0,1,450000,2019,10,8
6997,6997,0,0,Not specified,0,0,Not specified,0,1,1,...,0,SantiagoDeSurco,Not specified,Not specified,Not specified,1,450000,2019,9,16
6998,6998,Not specified,Not specified,1,0,Not specified,Not specified,Not specified,Not specified,Not specified,...,Not specified,Bellavista,0,Muy bueno,0,1,240000,2019,8,29


In [148]:

def remove_trash_data(column,dataset):
    allowed_values = ['0', '1', 'Nulo', 'Not specified', 'Si', 'No']
    print('--------------------------------')
    print('Removing trash data from column:', column)
    print(dataset.groupby(column).size())
    dataset[column] = dataset[column].apply(lambda x: x if x in allowed_values or pd.isna(x) else pd.NA)
    #dataset = dataset[(dataset[column].isin(allowed_values)) | (dataset[column].isnull())]
    print(dataset.groupby(column).size())
    return dataset

In [149]:
trash_columns = ['Internet', 'Sauna_Area', 'Air_Conditioning', 'Independent_Entrance', 'Jacuzzi', 'Service_Bathroom'
                 ,'Cable', 'Service_Room', 'Internet_Room', 'Daycare', 'Daily_Dining_Room', 'Heating', 'Pets']
for column in trash_columns:
    data = remove_trash_data(column, data)

--------------------------------
Removing trash data from column: Internet
Internet
0                                                                2105
1                                                                1194
Not specified                                                    3160
Panamericana Sur Km 94.5 Asia                                       1
Panamericana Sur Km 94.5 Club Playa Las Arenas Asia                 1
Panamericana Sur Km 94.5 Club Playa Las Arenas Casa B_22 Asia       1
Publicado el 05.09.19                                               1
Publicado el 10.09.19                                               1
Publicado el 12.08.19                                               2
Publicado el 13.04.19                                               5
Publicado el 13.09.19                                               1
Publicado el 14.09.19                                               1
Publicado el 17.09.19                                               6
Public

In [150]:
data

,Id,Gas_Connection,Fireplace,Entrance_Hall,Kitchenette,Equipped,Garage_Type,Furnished,Drainage,Telephone,...,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom,Price,Publication_Year,Publication_Month,Publication_Day
0,0,1,Not specified,0,0,Not specified,Paralelas,Not specified,Not specified,0,...,NaN,NaN,NaN,NaN,NaN,NaN,210000,2019,9,18
1,1,0,1,Not specified,0,0,Paralelas,0,0,0,...,0,LaMolina,Not specified,Bueno,Not specified,1,660000,2019,4,9
2,2,Not specified,Not specified,1,0,Not specified,Lineales,Not specified,Not specified,Not specified,...,Not specified,LaMolina,0,Bueno,0,1,1500000,2019,1,8
3,3,0,1,Not specified,0,0,Separadas,0,0,0,...,0,SanIsidro,Not specified,Remodelado,Not specified,1,1100000,2019,12,8
4,4,0,Not specified,Not specified,0,Not specified,Lineales,Not specified,Not specified,0,...,Not specified,VillaMariaDelTriunfo,Not specified,Regular,Not specified,0,110000,2019,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,6995,Not specified,Not specified,Not specified,0,Not specified,Not specified,Not specified,Not specified,Not specified,...,Not specified,LaVictoria,Not specified,Bueno,Not specified,0,450000,2019,7,26
6996,6996,0,0,1,0,0,Not specified,1,0,0,...,0,Asia,0,Not specified,0,1,450000,2019,10,8
6997,6997,0,0,Not specified,0,0,Not specified,0,1,1,...,0,SantiagoDeSurco,Not specified,Not specified,Not specified,1,450000,2019,9,16
6998,6998,Not specified,Not specified,1,0,Not specified,Not specified,Not specified,Not specified,Not specified,...,Not specified,Bellavista,0,Muy bueno,0,1,240000,2019,8,29


In [151]:
data = data[data['Age']!= 'Not specified']
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')

In [152]:
import numpy as np
def replace_not_specified_with_mode(column, dataset):
    dataset[column] = dataset[column].apply(lambda x: 0 if pd.isna(x) else
                                            2 if (x == '1' or x == 'Si') else 
                                            1 if (x == '0' or x == 'No') else 0 )
    #dataset[column+'_missing'] = dataset[column].isna().astype(int)
    dataset[column].fillna(0, inplace=True)
    #dataset[column].fillna(False, inplace=True)

    print(dataset.groupby(column).size())
    return dataset


In [153]:
# Posibles columnas repetidas en el dataset
# Sauna, Sauna_area,
# Oceanfront, Near_Sea
# Kitchenette, Kitchen_with_Cabinets


columns_replace_with_mode = ['Gas_Connection', 'Fireplace', 'Entrance_Hall', 'Kitchenette',
           'Equipped', 'Furnished', 'Drainage', 'Telephone', 'Guest_Bathroom', 'BBQ_Area',
           'Living_Room', 'Nearby_Parks', 'Solarium', 'Commercial_Use', 'Internal_Garden', 
           'Electricity', 'Patio', 'Children_Playground', 'Green_Areas', 'Electric_Doorman',
           'Intercom', 'Near_Sea', 'Sauna', 'Cinema_Room', 'Cleaning_Service', 'Terrace',
           'Sports_Area', 'Security_System', 'Water_Heater', 'Professional_Use', 'Club_House',
           'Internal_Park', 'Laundry_Room', 'Nearby_Schools', 'Balcony', 'Attic', 'Oceanfront', 'Security_Guard',
           'Swimming_Pool', 'Electric_Fence', 'Air_Conditioning', 'Hall', 'Nearby_Shopping_Centers', 'Kitchen',
           'Water', 'Basement', 'Independent_Bathroom', 'Walk_in_Closet', 'Grill', 'Closet',
           'Internet', 'Sauna_Area', 'Kitchen_with_Cabinets', 'Gym', 'Handicap_Access', 'Dining_Room',
           'Office', 'Service_Bathroom', 'Storage_Room', 'Cable', 'Jacuzzi', 'Independent_Entrance', 'Service_Room',
           'Internet_Room', 'Daycare', 'Daily_Dining_Room', 'Heating', 'Match', 'Pets', 'Garden']
for column in columns_replace_with_mode:
    print(f'Processing column: {column}')
    data = replace_not_specified_with_mode(column, data)

Processing column: Gas_Connection
Gas_Connection
0    3341
1    3175
2     484
dtype: int64
Processing column: Fireplace
Fireplace
0    2411
1    3270
2    1319
dtype: int64
Processing column: Entrance_Hall
Entrance_Hall
0    2441
1    1864
2    2695
dtype: int64
Processing column: Kitchenette
Kitchenette
0     209
1    6791
dtype: int64
Processing column: Equipped
Equipped
0    2411
1    4161
2     428
dtype: int64
Processing column: Furnished
Furnished
0    2411
1    3756
2     833
dtype: int64
Processing column: Drainage
Drainage
0    2411
1    3293
2    1296
dtype: int64
Processing column: Telephone
Telephone
0    3341
1    2361
2    1298
dtype: int64
Processing column: Guest_Bathroom
Guest_Bathroom
0     209
1    3172
2    3619
dtype: int64
Processing column: BBQ_Area
BBQ_Area
0    2441
1    3749
2     810
dtype: int64
Processing column: Living_Room
Living_Room
0     209
1    3876
2    2915
dtype: int64
Processing column: Nearby_Parks
Nearby_Parks
0    4105
1     678
2    2217
dty

/tmp/ipykernel_35404/1713995184.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(0, inplace=True)
/tmp/ipykernel_35404/1713995184.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

Children_Playground
0    2441
1    4251
2     308
dtype: int64
Processing column: Green_Areas
Green_Areas
0    2441
1    3602
2     957
dtype: int64
Processing column: Electric_Doorman
Electric_Doorman
0    2411
1    3867
2     722
dtype: int64
Processing column: Intercom
Intercom
0    2411
1    3598
2     991
dtype: int64
Processing column: Near_Sea
Near_Sea
0    4105
1    2493
2     402
dtype: int64
Processing column: Sauna
Sauna
0    2411
1    4423
2     166
dtype: int64
Processing column: Cinema_Room
Cinema_Room
0    2441
1    4540
2      19
dtype: int64
Processing column: Cleaning_Service
Cleaning_Service
0    3341
1    3258
2     401
dtype: int64
Processing column: Terrace
Terrace
0     209
1    3120
2    3671
dtype: int64
Processing column: Sports_Area
Sports_Area
0    2441
1    4003
2     556
dtype: int64
Processing column: Security_System
Security_System
0    3341
1    2689
2     970
dtype: int64
Processing column: Water_Heater
Water_Heater
0    2788
1    3193
2    1019
dtype:

/tmp/ipykernel_35404/1713995184.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(0, inplace=True)
/tmp/ipykernel_35404/1713995184.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

Internet
0    3701
1    2105
2    1194
dtype: int64
Processing column: Sauna_Area
Sauna_Area
0    2710
1    4191
2      99
dtype: int64
Processing column: Kitchen_with_Cabinets
Kitchen_with_Cabinets
0    2782
1    2714
2    1504
dtype: int64
Processing column: Gym
Gym
0    2707
1    3919
2     374
dtype: int64
Processing column: Handicap_Access
Handicap_Access
0    4369
1    2223
2     408
dtype: int64
Processing column: Dining_Room
Dining_Room
0     691
1     557
2    5752
dtype: int64
Processing column: Office
Office
0     687
1    4223
2    2090
dtype: int64
Processing column: Service_Bathroom
Service_Bathroom
0     720
1    1400
2    4880
dtype: int64
Processing column: Storage_Room
Storage_Room
0     717
1    4828
2    1455
dtype: int64
Processing column: Cable
Cable
0    3698
1    1997
2    1305
dtype: int64
Processing column: Jacuzzi
Jacuzzi
0    2816
1    3330
2     854
dtype: int64
Processing column: Independent_Entrance
Independent_Entrance
0    4373
1    2624
2       3
dtype

/tmp/ipykernel_35404/1713995184.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(0, inplace=True)
/tmp/ipykernel_35404/1713995184.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

In [154]:
def clean_categorical_columns(dataset):
    categorical_cols = ['Garage_Type', 'Province', 'Type', 'Location', 'Natural_Light', 'Advertiser', 'Beach_Resort', 'District', 'Property_Condition']
    for column in categorical_cols:
        print(f'Processing categorical column: {column}')
        dataset[column] = dataset[column].astype('category')
        dataset[column] = dataset[column].fillna(dataset[column].mode()[0])
    return dataset


In [155]:
data = clean_categorical_columns(data)

Processing categorical column: Garage_Type
Processing categorical column: Province
Processing categorical column: Type
Processing categorical column: Location
Processing categorical column: Natural_Light
Processing categorical column: Advertiser
Processing categorical column: Beach_Resort
Processing categorical column: District
Processing categorical column: Property_Condition


In [156]:
# Drop unique columns
for column in data.columns:
    if len(data[column].unique()) == len(data):
        print(f'Dropping unique column: {column}')
        data.drop(columns=[column], inplace=True)
    elif len(data[column].unique()) == 1:
        print(f'Dropping column with one value: {column}')
        data.drop(columns=[column], inplace=True)

Dropping unique column: Id


Dropping column with one value: Match


In [157]:
data

,Gas_Connection,Fireplace,Entrance_Hall,Kitchenette,Equipped,Garage_Type,Furnished,Drainage,Telephone,Guest_Bathroom,...,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom,Price,Publication_Year,Publication_Month,Publication_Day
0,2,0,1,1,0,Paralelas,0,0,1,1,...,0,LaMolina,0,Not specified,0,0,210000,2019,9,18
1,1,2,0,1,1,Paralelas,1,1,1,2,...,1,LaMolina,0,Bueno,0,2,660000,2019,4,9
2,0,0,2,1,0,Lineales,0,0,0,1,...,0,LaMolina,1,Bueno,1,2,1500000,2019,1,8
3,1,2,0,1,1,Separadas,1,1,1,2,...,1,SanIsidro,0,Remodelado,0,2,1100000,2019,12,8
4,1,0,0,1,0,Lineales,0,0,1,1,...,0,VillaMariaDelTriunfo,0,Regular,0,1,110000,2019,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0,0,0,1,0,Not specified,0,0,0,1,...,0,LaVictoria,0,Bueno,0,1,450000,2019,7,26
6996,1,1,2,1,1,Not specified,2,1,1,2,...,1,Asia,1,Not specified,1,2,450000,2019,10,8
6997,1,1,0,1,1,Not specified,1,2,2,2,...,1,SantiagoDeSurco,0,Not specified,0,2,450000,2019,9,16
6998,0,0,2,1,0,Not specified,0,0,0,2,...,0,Bellavista,1,Muy bueno,1,2,240000,2019,8,29


In [158]:
# Identify categorical and numerical columns
columns_object =data.select_dtypes(include=['object']).columns.tolist()
print(columns_object)
data.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()
#data.drop(columns=columns_object, inplace=True)

[]


['Gas_Connection',
 'Fireplace',
 'Entrance_Hall',
 'Kitchenette',
 'Equipped',
 'Furnished',
 'Drainage',
 'Telephone',
 'Guest_Bathroom',
 'BBQ_Area',
 'Living_Room',
 'Nearby_Parks',
 'Solarium',
 'Commercial_Use',
 'Internal_Garden',
 'Garages',
 'Electricity',
 'Patio',
 'Children_Playground',
 'Number_Bathrooms',
 'Green_Areas',
 'Electric_Doorman',
 'Construction_Area_m2',
 'Intercom',
 'Near_Sea',
 'Sauna',
 'Cinema_Room',
 'Cleaning_Service',
 'Terrace',
 'Sports_Area',
 'Security_System',
 'Water_Heater',
 'Professional_Use',
 'Club_House',
 'Internal_Park',
 'Laundry_Room',
 'Nearby_Schools',
 'Balcony',
 'Attic',
 'Oceanfront',
 'Security_Guard',
 'Swimming_Pool',
 'Electric_Fence',
 'Air_Conditioning',
 'Hall',
 'Nearby_Shopping_Centers',
 'Kitchen',
 'Water',
 'Basement',
 'Independent_Bathroom',
 'Walk_in_Closet',
 'Age',
 'Number_Floors',
 'Daily_Dining_Room',
 'Grill',
 'Closet',
 'Internet',
 'Sauna_Area',
 'Bedrooms',
 'Pets',
 'Kitchen_with_Cabinets',
 'Service_Room

In [159]:
data.select_dtypes(include=['int64', 'float64']).columns.tolist()

['Gas_Connection',
 'Fireplace',
 'Entrance_Hall',
 'Kitchenette',
 'Equipped',
 'Furnished',
 'Drainage',
 'Telephone',
 'Guest_Bathroom',
 'BBQ_Area',
 'Living_Room',
 'Nearby_Parks',
 'Solarium',
 'Commercial_Use',
 'Internal_Garden',
 'Garages',
 'Electricity',
 'Patio',
 'Children_Playground',
 'Number_Bathrooms',
 'Green_Areas',
 'Electric_Doorman',
 'Construction_Area_m2',
 'Intercom',
 'Near_Sea',
 'Sauna',
 'Cinema_Room',
 'Cleaning_Service',
 'Terrace',
 'Sports_Area',
 'Security_System',
 'Water_Heater',
 'Professional_Use',
 'Club_House',
 'Internal_Park',
 'Laundry_Room',
 'Nearby_Schools',
 'Balcony',
 'Attic',
 'Oceanfront',
 'Security_Guard',
 'Swimming_Pool',
 'Electric_Fence',
 'Air_Conditioning',
 'Hall',
 'Nearby_Shopping_Centers',
 'Kitchen',
 'Water',
 'Basement',
 'Independent_Bathroom',
 'Walk_in_Closet',
 'Age',
 'Number_Floors',
 'Daily_Dining_Room',
 'Grill',
 'Closet',
 'Internet',
 'Sauna_Area',
 'Bedrooms',
 'Pets',
 'Kitchen_with_Cabinets',
 'Service_Room

In [160]:
import numpy as np
def replace_not_specified_with_mediam(column, dataset):
    dataset[column] = dataset[column].apply(lambda x: np.nan if (x == 'Not specified' or x == 'Nulo') else x)
    dataset[column].fillna(dataset[column].median(), inplace=True)
    return dataset

In [161]:
data = replace_not_specified_with_mediam('Total_Area_m2', data)
data.drop_duplicates(inplace=True)

/tmp/ipykernel_35404/4007228366.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(dataset[column].median(), inplace=True)


In [162]:
data.drop(columns=['Beach_Resort'], inplace=True)

In [163]:
data

,Gas_Connection,Fireplace,Entrance_Hall,Kitchenette,Equipped,Garage_Type,Furnished,Drainage,Telephone,Guest_Bathroom,...,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom,Price,Publication_Year,Publication_Month,Publication_Day
0,2,0,1,1,0,Paralelas,0,0,1,1,...,0,LaMolina,0,Not specified,0,0,210000,2019,9,18
1,1,2,0,1,1,Paralelas,1,1,1,2,...,1,LaMolina,0,Bueno,0,2,660000,2019,4,9
2,0,0,2,1,0,Lineales,0,0,0,1,...,0,LaMolina,1,Bueno,1,2,1500000,2019,1,8
3,1,2,0,1,1,Separadas,1,1,1,2,...,1,SanIsidro,0,Remodelado,0,2,1100000,2019,12,8
4,1,0,0,1,0,Lineales,0,0,1,1,...,0,VillaMariaDelTriunfo,0,Regular,0,1,110000,2019,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0,0,0,1,0,Not specified,0,0,0,1,...,0,LaVictoria,0,Bueno,0,1,450000,2019,7,26
6996,1,1,2,1,1,Not specified,2,1,1,2,...,1,Asia,1,Not specified,1,2,450000,2019,10,8
6997,1,1,0,1,1,Not specified,1,2,2,2,...,1,SantiagoDeSurco,0,Not specified,0,2,450000,2019,9,16
6998,0,0,2,1,0,Not specified,0,0,0,2,...,0,Bellavista,1,Muy bueno,1,2,240000,2019,8,29


In [164]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6966 entries, 0 to 6999
Data columns (total 88 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   Gas_Connection           6966 non-null   int64   
 1   Fireplace                6966 non-null   int64   
 2   Entrance_Hall            6966 non-null   int64   
 3   Kitchenette              6966 non-null   int64   
 4   Equipped                 6966 non-null   int64   
 5   Garage_Type              6966 non-null   category
 6   Furnished                6966 non-null   int64   
 7   Drainage                 6966 non-null   int64   
 8   Telephone                6966 non-null   int64   
 9   Guest_Bathroom           6966 non-null   int64   
 10  BBQ_Area                 6966 non-null   int64   
 11  Living_Room              6966 non-null   int64   
 12  Nearby_Parks             6966 non-null   int64   
 13  Solarium                 6966 non-null   int64   
 14  Commercial_Us

In [165]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Select features and target
X = data.drop(columns=['Price'])
y = data['Price']
y = np.log(y)  # Log-transform the target variable for better performance

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['category']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int', 'float']).columns.tolist()
numerical_cols.remove('Construction_Area_m2')
numerical_cols.remove('Total_Area_m2')
log_cols = ['Construction_Area_m2', 'Total_Area_m2']
boolean_cols = X.select_dtypes(include=['bool']).columns.tolist()

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('bool', LabelEncoder() ,boolean_cols),
        ('num', StandardScaler(), numerical_cols),
        ('log1', FunctionTransformer(np.log), ['Construction_Area_m2']),
        ('log2', FunctionTransformer(np.log1p), ['Total_Area_m2']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit and transform data
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Build neural network
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_processed.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


optimizer = Adam(clipvalue=0.001)  # Clip gradients to a value between -1 and 1
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Train model
history = model.fit(X_train_processed, y_train, epochs=150, batch_size=64, validation_split=0.1, callbacks=[early_stop], verbose=1)

# Evaluate model
loss, mae = model.evaluate(X_test_processed, y_test, verbose=0)
print(f"Test MAE: {mae:.2f}")

/home/noah/Projects/PUCP_machine-learning_garment-productivity/myenv/lib/python3.13/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 32.2226 - mae: 3.7797 - val_loss: 0.6237 - val_mae: 0.6441
Epoch 2/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3407 - mae: 0.4525 - val_loss: 0.3175 - val_mae: 0.4367
Epoch 3/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.2441 - mae: 0.3795 - val_loss: 0.3509 - val_mae: 0.4624
Epoch 4/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2070 - mae: 0.3492 - val_loss: 0.3151 - val_mae: 0.4319
Epoch 5/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1888 - mae: 0.3321 - val_loss: 0.2088 - val_mae: 0.3370
Epoch 6/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1676 - mae: 0.3138 - val_loss: 0.2843 - val_mae: 0.3919
Epoch 7/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1642 - mae: 0.3074 - val_loss: 0.2965 - val_mae: 0.4129
Epoch 8/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1516 - mae: 0.2940 - val_loss: 0.2513 - val_mae: 0.3714
Epoch 9/150
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1

In [166]:
import tensorflow as tf
from tensorflow.keras import layers, models
from xgboost import XGBRegressor

input_dim = X_train_processed.shape[1]
encoding_dim = 20  # compress to 4 features

# Build autoencoder model
input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(8, activation='relu')(input_layer)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)  # bottleneck

decoded = layers.Dense(8, activation='relu')(encoded)
decoded = layers.Dense(input_dim, activation='linear')(decoded)

autoencoder = models.Model(inputs=input_layer, outputs=decoded)

# Compile and train
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train_processed, X_train_processed, epochs=100, batch_size=32, shuffle=True, validation_data=(X_test_processed, X_test_processed), verbose=0)


TypeError: Expected float32, but got SparseTensor(indices=Tensor("data_3:0", shape=(None, 2), dtype=int64), values=Tensor("data_4:0", shape=(None,), dtype=float32), dense_shape=Tensor("data_5:0", shape=(2,), dtype=int64)) of type 'SparseTensor'.

In [ ]:
encoder = models.Model(inputs=input_layer, outputs=encoded)

# Encode training and test data
X_train_encoded = encoder.predict(X_train_processed)
X_test_encoded = encoder.predict(X_test_processed)

153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [ ]:

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
# Create the model
mlp = MLPRegressor(hidden_layer_sizes=(50, 30), max_iter=10000, random_state=1)

# Train the model
mlp.fit(X_train_processed, y_train)
y_pred = mlp.predict(X_test_processed).flatten()

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print(f"R2 score on test set: {r2:.4f}")

R2 score on test set: 0.6617


In [ ]:
import numpy as np
from sklearn.linear_model import Lasso

# Create Lasso model with regularization strength (alpha)
lasso = Lasso(alpha=0.1)

# Fit to training data
lasso.fit(X_train_processed, y_train)

y_pred = lasso.predict(X_test_processed).flatten()

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print(f"R2 score on test set: {r2:.4f}")

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.1,
    max_depth=10,
    random_state=151
)

# Train the model
xgb_model.fit(X_train_processed, y_train)

y_pred = xgb_model.predict(X_test_processed).flatten()

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print(f"R2 score on test set: {r2:.4f}")

R2 score on test set: 0.8285


In [ ]:
from sklearn.metrics import r2_score

# Predict on test set
y_pred = model.predict(X_test_processed).flatten()

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print(f"R2 score on test set: {r2:.4f}")

66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
R2 score on test set: 0.6862


In [167]:
test_data = pd.read_csv('test.csv')
test_data.columns = test_data.columns.str.replace("'", "", regex=False)
test_data.head()



,Id,Gas_Connection,Fireplace,Entrance_Hall,Kitchenette,Equipped,Garage_Type,Furnished,Drainage,Telephone,...,Handicap_Access,Dining_Room,Office,Total_Area_m2,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom
0,0,Not specified,0,1,0,0,Paralelas,0,0,Not specified,...,Not specified,1,1,340,0,LaMolina,0,Not specified,0,1
1,1,Not specified,Not specified,1,0,Not specified,Lineales,Not specified,Not specified,Not specified,...,Not specified,1,0,1138,Not specified,LaMolina,0,Bueno,0,1
2,2,Not specified,0,0,0,0,Paralelas,0,0,Not specified,...,Not specified,0,1,1353,0,LaMolina,0,Bueno,0,1
3,3,0,1,0,0,0,Paralelas,0,0,0,...,Not specified,1,0,230,0,SantiagoDeSurco,0,Muy bueno,0,1
4,4,0,Not specified,Not specified,0,Not specified,Not specified,Not specified,Not specified,0,...,Not specified,1,0,305,Not specified,LaMolina,Not specified,Not specified,Not specified,1


In [ ]:
# Remove Construction_Area column, its redundant with construction area
test_data.drop(columns=['Construction_Area'], inplace=True)
test_data = extract_date(test_data)
test_data['Total_Area_m2'] = pd.to_numeric(test_data['Total_Area_m2'], errors='coerce')
test_data = replace_not_specified_with_mediam('Total_Area_m2', test_data)
test_data = preprocess_data(test_data)
for column in numeric_columns:
    test_data = fill_numeric_column(column, test_data)

test_data['Age'] = pd.to_numeric(test_data['Age'], errors='coerce')

for column in trash_columns:
    test_data = remove_trash_data(column, test_data)
for column in columns_replace_with_mode:
    print(f'Processing column: {column}')
    test_data = replace_not_specified_with_mode(column, test_data)

test_data = clean_categorical_columns(test_data)
    
# Drop unique columns
for column in test_data.columns:
    if len(test_data[column].unique()) == len(test_data):
        print(f'Dropping unique column: {column}')
        test_data.drop(columns=[column], inplace=True)
    elif len(test_data[column].unique()) == 1:
        print(f'Dropping column with one value: {column}')
        test_data.drop(columns=[column], inplace=True)

test_data.drop(columns=['Beach_Resort'], inplace=True)
test_data.info()

/tmp/ipykernel_35404/2842108097.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_set['Publication_Date'] = pd.to_datetime(data_set['Publication_Date'], errors='coerce')
/tmp/ipykernel_35404/4007228366.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(dataset[column].median(), inplace=True)
/tmp/ipykernel_35404/2115012877.py:12: FutureWarning: A value is try

--------------------------------
Removing trash data from column: Internet
Internet
0                192
1                112
Not specified    309
dtype: int64
Internet
0                192
1                112
Not specified    309
dtype: int64
--------------------------------
Removing trash data from column: Sauna_Area
Sauna_Area
0                404
1                  8
Not specified    201
dtype: int64
Sauna_Area
0                404
1                  8
Not specified    201
dtype: int64
--------------------------------
Removing trash data from column: Air_Conditioning
Air_Conditioning
0                317
1                 72
Not specified    224
dtype: int64
Air_Conditioning
0                317
1                 72
Not specified    224
dtype: int64
--------------------------------
Removing trash data from column: Independent_Entrance
Independent_Entrance
0                249
Not specified    364
dtype: int64
Independent_Entrance
0                249
Not specified    364
dtype: in

/tmp/ipykernel_35404/1713995184.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[column].fillna(0, inplace=True)
/tmp/ipykernel_35404/1713995184.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

In [ ]:
test_data

,Gas_Connection,Fireplace,Entrance_Hall,Kitchenette,Equipped,Garage_Type,Furnished,Drainage,Telephone,Guest_Bathroom,...,Total_Area_m2,Heating,District,Daycare,Property_Condition,Internet_Room,Service_Bathroom,Publication_Year,Publication_Month,Publication_Day
0,0,1,2,1,1,Paralelas,1,1,0,1,...,340.0,1,LaMolina,1,Not specified,1,2,2019,8,20
1,0,0,2,1,0,Lineales,0,0,0,2,...,1138.0,0,LaMolina,1,Bueno,1,2,2019,8,22
2,0,1,1,1,1,Paralelas,1,1,0,1,...,1353.0,1,LaMolina,1,Bueno,1,2,2019,9,16
3,1,2,1,1,1,Paralelas,1,1,1,1,...,230.0,1,SantiagoDeSurco,1,Muy bueno,1,2,2019,8,20
4,1,0,0,1,0,Not specified,0,0,1,2,...,305.0,0,LaMolina,0,Not specified,0,2,2019,9,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,0,0,2,1,0,Paralelas,0,0,0,1,...,600.0,0,LaMolina,1,Not specified,1,2,2019,8,9
653,2,1,2,1,2,Separadas,2,1,1,2,...,204.0,1,Asia,1,Excelente,1,2,2019,8,29
654,1,0,2,1,0,Paralelas,0,0,1,2,...,270.0,0,LaMolina,1,Not specified,1,2,2019,8,9
655,1,1,1,1,1,Paralelas,1,1,1,1,...,160.0,1,Asia,1,Not specified,1,2,2019,7,25


In [ ]:
# Identify categorical and numerical columns
#columns_object =test_data.select_dtypes(include=['object']).columns.tolist()
print(columns_object)
data.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()
#test_data.drop(columns=columns_object, inplace=True)

[]


['Gas_Connection',
 'Fireplace',
 'Entrance_Hall',
 'Kitchenette',
 'Equipped',
 'Furnished',
 'Drainage',
 'Telephone',
 'Guest_Bathroom',
 'BBQ_Area',
 'Living_Room',
 'Nearby_Parks',
 'Solarium',
 'Commercial_Use',
 'Internal_Garden',
 'Garages',
 'Electricity',
 'Patio',
 'Children_Playground',
 'Number_Bathrooms',
 'Green_Areas',
 'Electric_Doorman',
 'Construction_Area_m2',
 'Intercom',
 'Near_Sea',
 'Sauna',
 'Cinema_Room',
 'Cleaning_Service',
 'Terrace',
 'Sports_Area',
 'Security_System',
 'Water_Heater',
 'Professional_Use',
 'Club_House',
 'Internal_Park',
 'Laundry_Room',
 'Nearby_Schools',
 'Balcony',
 'Attic',
 'Oceanfront',
 'Security_Guard',
 'Swimming_Pool',
 'Electric_Fence',
 'Air_Conditioning',
 'Hall',
 'Nearby_Shopping_Centers',
 'Kitchen',
 'Water',
 'Basement',
 'Independent_Bathroom',
 'Walk_in_Closet',
 'Age',
 'Number_Floors',
 'Daily_Dining_Room',
 'Grill',
 'Closet',
 'Internet',
 'Sauna_Area',
 'Bedrooms',
 'Pets',
 'Kitchen_with_Cabinets',
 'Service_Room

In [ ]:
data.columns

Index(['Gas_Connection', 'Fireplace', 'Entrance_Hall', 'Kitchenette',
       'Equipped', 'Garage_Type', 'Furnished', 'Drainage', 'Telephone',
       'Guest_Bathroom', 'BBQ_Area', 'Living_Room', 'Nearby_Parks', 'Solarium',
       'Commercial_Use', 'Province', 'Internal_Garden', 'Garages',
       'Electricity', 'Patio', 'Children_Playground', 'Type',
       'Number_Bathrooms', 'Green_Areas', 'Electric_Doorman',
       'Construction_Area_m2', 'Intercom', 'Near_Sea', 'Sauna', 'Cinema_Room',
       'Cleaning_Service', 'Terrace', 'Sports_Area', 'Security_System',
       'Location', 'Water_Heater', 'Professional_Use', 'Club_House',
       'Internal_Park', 'Laundry_Room', 'Nearby_Schools', 'Balcony', 'Attic',
       'Oceanfront', 'Security_Guard', 'Natural_Light', 'Swimming_Pool',
       'Electric_Fence', 'Advertiser', 'Air_Conditioning', 'Hall',
       'Nearby_Shopping_Centers', 'Kitchen', 'Water', 'Basement',
       'Independent_Bathroom', 'Walk_in_Closet', 'Age', 'Number_Floors',
       'Da

In [170]:
test_data.columns

Index(['Gas_Connection', 'Fireplace', 'Entrance_Hall', 'Kitchenette',
       'Equipped', 'Garage_Type', 'Furnished', 'Drainage', 'Telephone',
       'Guest_Bathroom', 'BBQ_Area', 'Living_Room', 'Nearby_Parks', 'Solarium',
       'Commercial_Use', 'Province', 'Internal_Garden', 'Garages',
       'Electricity', 'Patio', 'Children_Playground', 'Type',
       'Number_Bathrooms', 'Green_Areas', 'Electric_Doorman',
       'Construction_Area_m2', 'Intercom', 'Near_Sea', 'Sauna', 'Cinema_Room',
       'Cleaning_Service', 'Terrace', 'Sports_Area', 'Security_System',
       'Location', 'Water_Heater', 'Professional_Use', 'Club_House',
       'Internal_Park', 'Laundry_Room', 'Nearby_Schools', 'Balcony', 'Attic',
       'Oceanfront', 'Security_Guard', 'Natural_Light', 'Swimming_Pool',
       'Electric_Fence', 'Advertiser', 'Air_Conditioning', 'Hall',
       'Nearby_Shopping_Centers', 'Kitchen', 'Water', 'Basement',
       'Independent_Bathroom', 'Walk_in_Closet', 'Age', 'Number_Floors',
       'Da

In [ ]:
test_data_processed = preprocessor.transform(test_data)
# Predict prices
predicted_prices = xgb_model.predict(test_data_processed)
print(predicted_prices)
predicted_prices = np.exp(predicted_prices)  # Inverse log transformation to get actual prices
# Add predictions to test_data
test_data['Predicted_Price'] = predicted_prices.flatten()
test_data[['Predicted_Price']].head()

df = pd.DataFrame(predicted_prices, columns=['Price'])
df.index.name = 'Id'
print(df)
df.to_csv('submission.csv', index=True)

[13.583468  13.63192   13.973776  12.809189  13.302343  12.074089
 13.250003  13.318387  12.192914  13.310146  13.336928  11.98727
 13.593143  13.666144  14.143209  13.353835  13.182859  13.770373
 13.032463  11.768638  12.996999  12.297492  13.725607  13.498281
 12.813213  13.983319  12.502188  13.585258  13.118653  12.669609
 12.35134   13.331536  13.038154  13.0561075 12.538413  13.963745
 13.938547  12.369761  12.995189  12.464429  12.904414  12.717625
 12.924998  11.593926  13.958565  13.367903  13.281977  13.580219
 14.119409  11.969377  13.9299135 11.737344  14.114856  14.164799
 12.14845   12.676369  14.132932  13.902496  12.651899  13.10104
 13.664648  12.756068  12.353248  14.599238  13.957153  14.053106
 12.983976  13.3026    12.997943  12.851238  14.22883   13.131636
 13.978355  12.968697  13.357662  12.136291  13.585445  14.210873
 12.293147  11.8538    13.691772  12.596718  13.152158  13.1060295
 12.098606  12.265519  14.068237  11.760906  13.904015  13.382946
 13.364503 